## Data Warehouse

### 0. Create Mock Datas

In [21]:
import pandas as pd
import random
import string
from datetime import datetime, timedelta

# store list
STORES = {
    "IM001": "INDOMARET LATIGO SQUARE",
    "IM002": "INDOMARET ANIVA JUNCTION",
    "IM003": "INDOMARET FRESH BOSTON SQUARE",
}

STORE_IDS = list(STORES.keys())

# Suppliers
SUPPLIERS = {
    "SUP001": "Indofood",
    "SUP002": "Danone",
    "SUP003": "Le Minerale",
    "SUP004": "Sosro",
    "SUP005": "Mayora",
    "SUP006": "Unilever"
}

# product list (item name, supplier_id, category, price)
PRODUCTS = {
    "P001": ("Indomie Goreng", "SUP001", "Food", 3500),
    "P002": ("Indomie Kari", "SUP001", "Food", 3500),
    "P003": ("Pop Mie Ayam", "SUP001", "Food", 5500),
    "P004": ("Bimoli Minyak Goreng 1L", "SUP001", "Food", 25000),

    "P005": ("Aqua 600ml", "SUP002", "Beverage", 3500),
    "P006": ("Aqua 1.5L", "SUP002", "Beverage", 7000),

    "P007": ("Le Minerale 600ml", "SUP003", "Beverage", 3500),
    
    "P008": ("Teh Botol Sosro", "SUP004", "Beverage", 5000),
    "P009": ("S-Tee Lemon", "SUP004", "Beverage", 4500),
    
    "P010": ("SilverQueen Chocolate", "SUP005", "Candy/Snack", 12000),
    "P011": ("Kopiko Coffee Candy", "SUP005", "Candy/Snack", 5000),
    "P012": ("Roma Marie", "SUP005", "Candy/Snack", 6000),

    "P013": ("Sunlight Dish Soap 1L", "SUP006", "Household", 20000),
    "P014": ("Lifebouy Sabun Mandi", "SUP006", "Personal Care", 8000),
    "P015": ("Clear Shampoo 170ml", "SUP006", "Personal Care", 25000)
}

ITEM_IDS = list(PRODUCTS.keys())

# payment methods
PAYMENTS = {
    "PM01": "Cash",
    "PM02": "Debit",
    "PM03": "Credit",
    "PM04": "OVO",
    "PM05": "GoPay",
    "PM06": "ShopeePay"
}

PAYMENT_IDS = list(PAYMENTS.keys())

# cashier list
CASHIERS = [f"L{str(i).zfill(3)}" for i in range(1, 10)]
STORE_CASHIERS = {
    "IM001": CASHIERS[:2],
    "IM002": CASHIERS[2:4],
    "IM003": CASHIERS[4:7],
}

store_cashier_index = {store: 0 for store in STORE_IDS}

# promo days
promo_ranges = [
    (1, 3), # 1-3 Nov
    (8, 10), # 8-10 Nov
    (11, 12), # 11-12 Nov
    (25, 28) # 25-28 Nov
]

def is_promo(day):
    for start, end in promo_ranges:
        if start <= day <= end:
            return True
    return False

# discount rules
DISCOUNT_RULES = {
    "Food": 0.10,
    "Beverage": 0.15,
    "Candy/Snack": 0.20,
    "Household": 0.05,
    "Personal Care": 0.10,
}

# ---------------------------------------------------------------------------
# CONFIG
# ---------------------------------------------------------------------------
START_DATE = datetime(2025, 11, 1)
END_DATE = datetime(2025, 11, 30)
TRANSACTION_PER_DAY = (10, 15)
ITEMS_PER_TRANSACTION = (3, 8)

# ---------------------------------------------------------------------------
# DATA GENERATION
# ---------------------------------------------------------------------------
rows = []
transaction_id_counter = 1
line_item_counter = 1

dates = [START_DATE + timedelta(days=i)
         for i in range((END_DATE - START_DATE).days + 1)]

for d in dates:
    num_transactions = random.randint(*TRANSACTION_PER_DAY)

    for _ in range(num_transactions):
        transaction_id = f"T{transaction_id_counter:05d}"
        customer_id = "C" + "".join(random.choices(string.digits, k=5))
        payment_method_id = random.choice(PAYMENT_IDS)
        store_id = random.choice(STORE_IDS)
        day = d.day

        cashier_list = STORE_CASHIERS[store_id]
        cashier_id = cashier_list[store_cashier_index[store_id] % len(cashier_list)]
        store_cashier_index[store_id] += 1

        num_items = random.randint(*ITEMS_PER_TRANSACTION)

        for _ in range(num_items):
            line_item_id = f"LI{line_item_counter:07d}"

            product_id = random.choice(ITEM_IDS)
            item_name, supplier_id, category, price = PRODUCTS[product_id]
            supplier_name = SUPPLIERS[supplier_id]

            quantity = random.randint(1, 5)
            
            promo_flag = 1 if is_promo(day) else 0
            discount = 0
            if promo_flag:
                discount_pct = DISCOUNT_RULES.get(category, 0)
                discount = int(quantity * price * discount_pct)

            total_price = (quantity * price) - discount

            rows.append([
                line_item_id, transaction_id, d.date(), customer_id, store_id,
                STORES[store_id], product_id, item_name, category,
                supplier_id, supplier_name,
                quantity, price, discount, total_price,
                payment_method_id, PAYMENTS[payment_method_id],
                cashier_id, promo_flag
            ])

            line_item_counter += 1

        transaction_id_counter += 1

### 1. Create DataFrame

In [22]:
columns = [
    "line_item_id", "transaction_id", "transaction_date", "customer_id", "store_id", 
    "store_name", "product_id", "item_name", "category",
    "supplier_id", "supplier_name",
    "quantity", "price", "discount", "total_price",
    "payment_method_id", "payment_method",
    "cashier_id", "promo_flag"
]

df = pd.DataFrame(rows, columns=columns)
df

,line_item_id,transaction_id,transaction_date,customer_id,store_id,store_name,product_id,item_name,category,supplier_id,supplier_name,quantity,price,discount,total_price,payment_method_id,payment_method,cashier_id,promo_flag
0,LI0000001,T00001,2025-11-01,C30641,IM001,INDOMARET LATIGO SQUARE,P007,Le Minerale 600ml,Beverage,SUP003,Le Minerale,4,3500,2100,11900,PM04,OVO,L001,1
1,LI0000002,T00001,2025-11-01,C30641,IM001,INDOMARET LATIGO SQUARE,P006,Aqua 1.5L,Beverage,SUP002,Danone,5,7000,5250,29750,PM04,OVO,L001,1
2,LI0000003,T00001,2025-11-01,C30641,IM001,INDOMARET LATIGO SQUARE,P001,Indomie Goreng,Food,SUP001,Indofood,1,3500,350,3150,PM04,OVO,L001,1
3,LI0000004,T00001,2025-11-01,C30641,IM001,INDOMARET LATIGO SQUARE,P006,Aqua 1.5L,Beverage,SUP002,Danone,1,7000,1050,5950,PM04,OVO,L001,1
4,LI0000005,T00002,2025-11-01,C34231,IM001,INDOMARET LATIGO SQUARE,P001,Indomie Goreng,Food,SUP001,Indofood,4,3500,1400,12600,PM01,Cash,L002,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2110,LI0002111,T00374,2025-11-30,C94076,IM001,INDOMARET LATIGO SQUARE,P003,Pop Mie Ayam,Food,SUP001,Indofood,3,5500,0,16500,PM01,Cash,L002,0
2111,LI0002112,T00374,2025-11-30,C94076,IM001,INDOMARET LATIGO SQUARE,P001,Indomie Goreng,Food,SUP001,Indofood,4,3500,0,14000,PM01,Cash,L002,0
2112,LI0002113,T00375,2025-11-30,C98626,IM002,INDOMARET ANIVA JUNCTION,P014,Lifebouy Sabun Mandi,Personal Care,SUP006,Unilever,5,8000,0,40000,PM02,Debit,L003,0
2113,LI0002114,T00375,2025-11-30,C98626,IM002,INDOMARET ANIVA JUNCTION,P007,Le Minerale 600ml,Beverage,SUP003,Le Minerale,4,3500,0,14000,PM02,Debit,L003,0


In [23]:
df.shape

(2115, 19)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2115 entries, 0 to 2114
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   line_item_id       2115 non-null   object
 1   transaction_id     2115 non-null   object
 2   transaction_date   2115 non-null   object
 3   customer_id        2115 non-null   object
 4   store_id           2115 non-null   object
 5   store_name         2115 non-null   object
 6   product_id         2115 non-null   object
 7   item_name          2115 non-null   object
 8   category           2115 non-null   object
 9   supplier_id        2115 non-null   object
 10  supplier_name      2115 non-null   object
 11  quantity           2115 non-null   int64 
 12  price              2115 non-null   int64 
 13  discount           2115 non-null   int64 
 14  total_price        2115 non-null   int64 
 15  payment_method_id  2115 non-null   object
 16  payment_method     2115 non-null   object


In [24]:
df.describe()

,quantity,price,discount,total_price,promo_flag
count,2115.000000,2115.000000,2115.000000,2115.000000,2115.000000
mean,3.014657,9045.626478,1287.612293,25761.087470,0.415130
std,1.423307,7362.628634,2330.278460,26500.807453,0.492861
min,1.000000,3500.000000,0.000000,2975.000000,0.000000
25%,2.000000,3500.000000,0.000000,9600.000000,0.000000
50%,3.000000,5500.000000,0.000000,16500.000000,0.000000
75%,4.000000,12000.000000,2000.000000,28000.000000,1.000000
max,5.000000,25000.000000,12500.000000,125000.000000,1.000000


In [26]:
df.promo_flag.value_counts()

promo_flag
0    1237
1     878
Name: count, dtype: int64

In [27]:
# convert to csv file
df.to_csv("sources/retail_transactions.csv", index=False)

# convert to parquet file
df.to_parquet("sources/retail_transactions.parquet", index=False)

In [29]:
new_df = pd.read_csv("sources/retail_transactions.csv")
new_df.head()

,line_item_id,transaction_id,transaction_date,customer_id,store_id,store_name,product_id,item_name,category,supplier_id,supplier_name,quantity,price,discount,total_price,payment_method_id,payment_method,cashier_id,promo_flag
0,LI0000001,T00001,2025-11-01,C30641,IM001,INDOMARET LATIGO SQUARE,P007,Le Minerale 600ml,Beverage,SUP003,Le Minerale,4,3500,2100,11900,PM04,OVO,L001,1
1,LI0000002,T00001,2025-11-01,C30641,IM001,INDOMARET LATIGO SQUARE,P006,Aqua 1.5L,Beverage,SUP002,Danone,5,7000,5250,29750,PM04,OVO,L001,1
2,LI0000003,T00001,2025-11-01,C30641,IM001,INDOMARET LATIGO SQUARE,P001,Indomie Goreng,Food,SUP001,Indofood,1,3500,350,3150,PM04,OVO,L001,1
3,LI0000004,T00001,2025-11-01,C30641,IM001,INDOMARET LATIGO SQUARE,P006,Aqua 1.5L,Beverage,SUP002,Danone,1,7000,1050,5950,PM04,OVO,L001,1
4,LI0000005,T00002,2025-11-01,C34231,IM001,INDOMARET LATIGO SQUARE,P001,Indomie Goreng,Food,SUP001,Indofood,4,3500,1400,12600,PM01,Cash,L002,1


In [16]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1963 entries, 0 to 1962
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   line_item_id       1963 non-null   object
 1   transaction_id     1963 non-null   object
 2   transaction_date   1963 non-null   object
 3   customer_id        1963 non-null   object
 4   store_id           1963 non-null   object
 5   store_name         1963 non-null   object
 6   product_id         1963 non-null   object
 7   item_name          1963 non-null   object
 8   category           1963 non-null   object
 9   supplier_id        1963 non-null   object
 10  supplier_name      1963 non-null   object
 11  quantity           1963 non-null   int64 
 12  price              1963 non-null   int64 
 13  discount           1963 non-null   int64 
 14  total_price        1963 non-null   int64 
 15  payment_method_id  1963 non-null   object
 16  payment_method     1963 non-null   object


In [30]:
new_df['transaction_date'] = pd.to_datetime(new_df['transaction_date'])

In [31]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2115 entries, 0 to 2114
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   line_item_id       2115 non-null   object        
 1   transaction_id     2115 non-null   object        
 2   transaction_date   2115 non-null   datetime64[ns]
 3   customer_id        2115 non-null   object        
 4   store_id           2115 non-null   object        
 5   store_name         2115 non-null   object        
 6   product_id         2115 non-null   object        
 7   item_name          2115 non-null   object        
 8   category           2115 non-null   object        
 9   supplier_id        2115 non-null   object        
 10  supplier_name      2115 non-null   object        
 11  quantity           2115 non-null   int64         
 12  price              2115 non-null   int64         
 13  discount           2115 non-null   int64         
 14  total_pr

###  2. Dimension Tables

In [32]:
# dim_date (Date Dimension)
dim_date = pd.DataFrame({
    "date": pd.date_range(new_df["transaction_date"].min(), new_df["transaction_date"].max())
})
dim_date["date_key"] = dim_date["date"].dt.strftime("%Y%m%d").astype(int)
dim_date["day"] = dim_date["date"].dt.day
dim_date["month"] = dim_date["date"].dt.month
dim_date["year"] = dim_date["date"].dt.year
dim_date["weekday"] = dim_date["date"].dt.weekday
dim_date["is_weekend"] = dim_date["weekday"].isin([5,6]).astype(int)

In [33]:
dim_date.head()

,date,date_key,day,month,year,weekday,is_weekend
0,2025-11-01,20251101,1,11,2025,5,1
1,2025-11-02,20251102,2,11,2025,6,1
2,2025-11-03,20251103,3,11,2025,0,0
3,2025-11-04,20251104,4,11,2025,1,0
4,2025-11-05,20251105,5,11,2025,2,0


In [34]:
# dim_store (Store Dimension)
dim_store = new_df[["store_id", "store_name"]].drop_duplicates().reset_index(drop=True)
dim_store = dim_store.sort_values("store_id").reset_index(drop=True)
dim_store.insert(0, "store_key", range(1, len(dim_store)+1))

In [35]:
dim_store

,store_key,store_id,store_name
0,1,IM001,INDOMARET LATIGO SQUARE
1,2,IM002,INDOMARET ANIVA JUNCTION
2,3,IM003,INDOMARET FRESH BOSTON SQUARE


In [36]:
# dim_product (Product Dimension)
dim_product = new_df[["product_id", "item_name", "category", "supplier_id"]].drop_duplicates()
dim_product = dim_product.sort_values("product_id").reset_index(drop=True)
dim_product.insert(0, "product_key", range(1, len(dim_product)+1))

In [37]:
dim_product

,product_key,product_id,item_name,category,supplier_id
0,1,P001,Indomie Goreng,Food,SUP001
1,2,P002,Indomie Kari,Food,SUP001
2,3,P003,Pop Mie Ayam,Food,SUP001
3,4,P004,Bimoli Minyak Goreng 1L,Food,SUP001
4,5,P005,Aqua 600ml,Beverage,SUP002
5,6,P006,Aqua 1.5L,Beverage,SUP002
6,7,P007,Le Minerale 600ml,Beverage,SUP003
7,8,P008,Teh Botol Sosro,Beverage,SUP004
8,9,P009,S-Tee Lemon,Beverage,SUP004
9,10,P010,SilverQueen Chocolate,Candy/Snack,SUP005


In [38]:
# dim_supplier(Supplier Dimension)
dim_supplier = new_df[["supplier_id", "supplier_name"]].drop_duplicates()
dim_supplier = dim_supplier.sort_values("supplier_id").reset_index(drop=True)
dim_supplier.insert(0, "supplier_key", range(1, len(dim_supplier)+1))

In [39]:
dim_supplier

,supplier_key,supplier_id,supplier_name
0,1,SUP001,Indofood
1,2,SUP002,Danone
2,3,SUP003,Le Minerale
3,4,SUP004,Sosro
4,5,SUP005,Mayora
5,6,SUP006,Unilever


In [40]:
# dim_customer (Customer Dimension)
dim_customer = new_df[["customer_id"]].drop_duplicates().reset_index(drop=True)
dim_customer.insert(0, "customer_key", range(1, len(dim_customer)+1))

In [41]:
dim_customer.head()

,customer_key,customer_id
0,1,C30641
1,2,C34231
2,3,C99723
3,4,C44242
4,5,C85209


In [42]:
# dim_cashier (Cashier Dimension)
dim_cashier = new_df[["cashier_id"]].drop_duplicates().reset_index(drop=True)
dim_cashier = dim_cashier.sort_values("cashier_id").reset_index(drop=True)
dim_cashier.insert(0, "cashier_key", range(1, len(dim_cashier)+1))

In [43]:
dim_cashier

,cashier_key,cashier_id
0,1,L001
1,2,L002
2,3,L003
3,4,L004
4,5,L005
5,6,L006
6,7,L007


In [44]:
# dim_payment (Payment Dimension)
dim_payment = new_df[["payment_method_id", "payment_method"]].drop_duplicates()
dim_payment = dim_payment.sort_values("payment_method_id").reset_index(drop=True)
dim_payment.insert(0, "payment_key", range(1, len(dim_payment)+1))

In [45]:
dim_payment

,payment_key,payment_method_id,payment_method
0,1,PM01,Cash
1,2,PM02,Debit
2,3,PM03,Credit
3,4,PM04,OVO
4,5,PM05,GoPay
5,6,PM06,ShopeePay


### 3. Fact Sales

In [46]:
fact_sales = new_df.merge(dim_date[["date_key", "date"]], left_on="transaction_date", right_on="date") \
               .merge(dim_store[["store_key", "store_id"]], on="store_id") \
               .merge(dim_product[["product_key", "product_id"]], on="product_id") \
               .merge(dim_supplier[["supplier_key", "supplier_id"]], on="supplier_id") \
               .merge(dim_customer[["customer_key", "customer_id"]], on="customer_id") \
               .merge(dim_cashier[["cashier_key", "cashier_id"]], on="cashier_id") \
               .merge(dim_payment[["payment_key", "payment_method_id"]], on="payment_method_id")

In [47]:
fact_sales = fact_sales[[
    "line_item_id",
    "transaction_id",
    "date_key",
    "store_key",
    "product_key",
    "customer_key",
    "cashier_key",
    "payment_key",
    "quantity",
    "price",
    "discount",
    "total_price",
    "promo_flag",
]]

In [48]:
fact_sales.head()

,line_item_id,transaction_id,date_key,store_key,product_key,customer_key,cashier_key,payment_key,quantity,price,discount,total_price,promo_flag
0,LI0000001,T00001,20251101,1,7,1,1,4,4,3500,2100,11900,1
1,LI0000002,T00001,20251101,1,6,1,1,4,5,7000,5250,29750,1
2,LI0000003,T00001,20251101,1,1,1,1,4,1,3500,350,3150,1
3,LI0000004,T00001,20251101,1,6,1,1,4,1,7000,1050,5950,1
4,LI0000005,T00002,20251101,1,1,2,2,1,4,3500,1400,12600,1


### 4. Derived Facts

In [54]:
# Derived Fact 1: Daily Sales (by Date)
fact_daily_sales = fact_sales.groupby("date_key").agg({
    "transaction_id": "nunique",
    "quantity": "sum",
    "total_price": "sum",
    "discount": "sum"
}).reset_index()

fact_daily_sales.columns = ["date_key", "transaction_count", "total_quantity",
                            "total_revenue", "total_discount"]

# Derived Metrics
fact_daily_sales["avg_transaction_value"] = (
    fact_daily_sales["total_revenue"] / fact_daily_sales["transaction_count"]
)

fact_daily_sales["avg_discount_per_transaction"] = (
    fact_daily_sales["total_discount"] / fact_daily_sales["transaction_count"]
)

In [52]:
fact_daily_sales.head()

,date_key,transaction_count,total_quantity,total_revenue,total_discount,avg_transaction_value,avg_discount_per_transaction
0,20251101,12,184,1479900,191600,123325.000000,15966.666667
1,20251102,12,211,1517100,220900,126425.000000,18408.333333
2,20251103,14,225,1631400,220600,116528.571429,15757.142857
3,20251104,13,233,1893500,0,145653.846154,0.000000
4,20251105,13,218,2364000,0,181846.153846,0.000000


In [55]:
# Derived Fact 2: Store Performance (by Store)
fact_store_performance = fact_sales.groupby("store_key").agg({
    "transaction_id": "nunique",
    "customer_key": "nunique",
    "total_price": "sum",
    "quantity": "sum"
}).reset_index()

fact_store_performance.columns = ["store_key", "transaction_count", 
                                  "customer_count", "total_revenue", "total_quantity"]

# Derived Metrics
fact_store_performance["avg_transaction_value"] = (
    fact_store_performance["total_revenue"] / fact_store_performance["transaction_count"]
)

fact_store_performance["avg_items_per_transaction"] = (
    fact_store_performance["total_quantity"] / fact_store_performance["transaction_count"]
)

In [56]:
fact_store_performance

,store_key,transaction_count,customer_count,total_revenue,total_quantity,avg_transaction_value,avg_items_per_transaction
0,1,130,130,19476575,2213,149819.807692,17.023077
1,2,121,121,17854550,2050,147558.264463,16.942149
2,3,124,124,17153575,2113,138335.282258,17.040323


In [57]:
# Derived Fact 3: Product Performance (by Product)
fact_product_performance = fact_sales.groupby("product_key").agg({
    "transaction_id": "nunique",
    "quantity": "sum",
    "total_price": "sum",
    "discount": "sum"
}).reset_index()

fact_product_performance.columns = ["product_key", "transaction_count", 
                                  "total_quantity_sold", "total_revenue", "total_discount"]

# Derived Metrics
fact_product_performance["avg_quantity_per_transaction"] = (
    fact_product_performance["total_quantity_sold"] / fact_product_performance["transaction_count"]
)

fact_product_performance["discount_rate"] = (
    fact_product_performance["total_discount"] / (fact_product_performance["total_revenue"] + fact_product_performance["total_discount"])
)

In [58]:
fact_product_performance

,product_key,transaction_count,total_quantity_sold,total_revenue,total_discount,avg_quantity_per_transaction,discount_rate
0,1,115,436,1466150,59850,3.791304,0.039220
1,2,123,434,1464050,54950,3.528455,0.036175
2,3,116,427,2253900,94600,3.681034,0.040281
3,4,114,405,9735000,390000,3.552632,0.038519
4,5,97,343,1124375,76125,3.536082,0.063411
5,6,123,428,2816450,179550,3.479675,0.059930
6,7,122,455,1480150,112350,3.729508,0.070549
7,8,116,417,1938750,146250,3.594828,0.070144
8,9,129,485,2042775,139725,3.759690,0.064021
9,10,113,420,4656000,384000,3.716814,0.076190


### 5. Semi-Additive Facts

In [59]:
# Semi-additive Fact 1: Daily Store Balance (by Date, Store)

fact_store_daily_balance = fact_sales.groupby(["date_key", "store_key"]).agg({
    "total_price": "sum"
}).reset_index()

fact_store_daily_balance.columns = ["date_key", "store_key", "daily_revenue"]

In [60]:
fact_store_daily_balance

,date_key,store_key,daily_revenue
0,20251101,1,310950
1,20251101,2,602950
2,20251101,3,566000
3,20251102,1,242850
4,20251102,2,925325
...,...,...,...
83,20251129,2,482500
84,20251129,3,206500
85,20251130,1,741000
86,20251130,2,619000


In [61]:
# Semi-additive Fact 2: Daily Product Inventory Movement (by Date, Product)

fact_product_daily_movement = fact_sales.groupby(["date_key", "product_key"]).agg({
    "quantity": "sum",
    "total_price": "sum"
}).reset_index()

fact_product_daily_movement.columns = ["date_key", "product_key", 
                                       "daily_quantity_sold", "daily_revenue"]

In [62]:
fact_product_daily_movement

,date_key,product_key,daily_quantity_sold,daily_revenue
0,20251101,1,21,66150
1,20251101,2,14,44100
2,20251101,3,12,59400
3,20251101,4,5,112500
4,20251101,5,7,20825
...,...,...,...,...
444,20251130,10,8,96000
445,20251130,11,7,35000
446,20251130,13,11,220000
447,20251130,14,31,248000


In [63]:
# Semi-additive Fact 3: Daily Customer Activity (by Date, Customer)

fact_customer_daily_activity = fact_sales.groupby(["date_key", "customer_key"]).agg({
    "transaction_id": "nunique",
    "quantity": "sum",
    "total_price": "sum"
}).reset_index()

fact_customer_daily_activity.columns = ["date_key", "customer_key", "daily_transactions",
                                       "daily_quantity", "daily_spend"]

In [64]:
fact_customer_daily_activity

,date_key,customer_key,daily_transactions,daily_quantity,daily_spend
0,20251101,1,1,11,50750
1,20251101,2,1,13,67825
2,20251101,3,1,23,273800
3,20251101,4,1,8,64300
4,20251101,5,1,18,112600
...,...,...,...,...,...
370,20251130,371,1,15,80000
371,20251130,372,1,29,185000
372,20251130,373,1,21,116000
373,20251130,374,1,22,181500


### 6. Factless Facts

In [70]:
# Factless Fact 1: Promo Coverage (by Date, Store)
fact_promo_coverage = fact_sales[fact_sales["promo_flag"] == 1][
    ["date_key", "store_key"]
].drop_duplicates().reset_index(drop=True)
fact_promo_coverage["event_type"] = "PROMO_ACTIVE"

In [74]:
fact_promo_coverage["date_key"].unique()

array([20251101, 20251102, 20251103, 20251108, 20251109, 20251110,
       20251111, 20251112, 20251125, 20251126, 20251127, 20251128])

In [75]:
# Factless_product_availability (by Date, Product)
fact_product_availability = fact_sales[["date_key", "product_key"]].drop_duplicates()
fact_product_availability = fact_product_availability.reset_index(drop=True)
fact_product_availability["availability_flag"] = 1

In [76]:
fact_product_availability

,date_key,product_key,availability_flag
0,20251101,7,1
1,20251101,6,1
2,20251101,1,1
3,20251101,4,1
4,20251101,5,1
...,...,...,...
444,20251130,15,1
445,20251130,2,1
446,20251130,11,1
447,20251130,5,1


### 7. Snapshot Facts

In [77]:
# Snapshot Fact 1: Daily Sales Snapshot (by Date)
fact_daily_snapshot = fact_sales.groupby("date_key").agg({
    "transaction_id": "nunique",
    "customer_key": "nunique",
    "quantity": "sum",
    "total_price": "sum",
    "discount": "sum",
}).reset_index()

fact_daily_snapshot.columns = ["date_key", "transaction_count", "customer_count",
                               "total_quantity", "total_revenue", "total_discount"]
fact_daily_snapshot["snapshot_type"] = "DAILY"

fact_daily_snapshot.head()

,date_key,transaction_count,customer_count,total_quantity,total_revenue,total_discount,snapshot_type
0,20251101,12,12,184,1479900,191600,DAILY
1,20251102,12,12,211,1517100,220900,DAILY
2,20251103,14,14,225,1631400,220600,DAILY
3,20251104,13,13,233,1893500,0,DAILY
4,20251105,13,13,218,2364000,0,DAILY


In [78]:
# Snapshot Fact 2: Daily Store Snapshot (by Date, Store)
fact_store_snapshot = fact_sales.groupby(["date_key", "store_key"]).agg({
    "transaction_id": "nunique",
    "customer_key": "nunique",
    "quantity": "sum",
    "total_price": "sum"
}).reset_index()

fact_store_snapshot.columns = ["date_key", "store_key", "transaction_count", 
                               "customer_count", "total_quantity", "total_revenue"]
fact_store_snapshot["snapshot_type"] = "DAILY_STORE"

fact_store_snapshot.head()

,date_key,store_key,transaction_count,customer_count,total_quantity,total_revenue,snapshot_type
0,20251101,1,4,4,54,310950,DAILY_STORE
1,20251101,2,4,4,73,602950,DAILY_STORE
2,20251101,3,4,4,57,566000,DAILY_STORE
3,20251102,1,3,3,54,242850,DAILY_STORE
4,20251102,2,6,6,103,925325,DAILY_STORE


### 8. Accumulating Snapshot Facts

In [81]:
# Accumulating Fact 1: Customer Lifetime Value (by Date, Customer)
df_sorted = new_df.sort_values("transaction_date").copy()
df_sorted["customer_cumulative_spend"] = df_sorted.groupby("customer_id")["total_price"].cumsum()
df_sorted["customer_transaction_count"] = df_sorted.groupby("customer_id").cumcount() + 1

# Merge with Dimension keys
fact_customer_accumulation = df_sorted.merge(
    dim_date[["date_key", "date"]], left_on="transaction_date", right_on="date"
).merge(
    dim_customer[["customer_key", "customer_id"]], on="customer_id"
)

fact_customer_accumulation = fact_customer_accumulation[[
    "date_key",
    "customer_key",
    "customer_cumulative_spend",
    "customer_transaction_count"
]].drop_duplicates(
    subset=["date_key", "customer_key"], 
    keep="last"
).reset_index(drop=True)

fact_customer_accumulation = fact_customer_accumulation.sort_values(
    ["customer_key", "date_key"]
).reset_index(drop=True)

In [82]:
fact_customer_accumulation

,date_key,customer_key,customer_cumulative_spend,customer_transaction_count
0,20251101,1,50750,4
1,20251101,2,67825,4
2,20251101,3,273800,7
3,20251101,4,64300,4
4,20251101,5,112600,8
...,...,...,...,...
370,20251130,371,80000,3
371,20251130,372,185000,8
372,20251130,373,116000,7
373,20251130,374,181500,7


In [85]:
# Accumulating Fact 2: Store Cumulative Performance (by Date, Store)
df_sorted_store = new_df.sort_values("transaction_date").copy()
df_sorted_store["store_cumulative_revenue"] = df_sorted_store.groupby("store_id")["total_price"].cumsum()
df_sorted_store["store_cumulative_transactions"] = df_sorted_store.groupby("store_id")["transaction_id"].transform(
    lambda x: (~x.duplicated()).cumsum()
)

# Merge with Dimension keys
fact_store_accumulation = df_sorted_store.merge(
    dim_date[["date_key", "date"]], left_on="transaction_date", right_on="date"
).merge(
    dim_store[["store_key", "store_id"]], on="store_id"
)

fact_store_accumulation = fact_store_accumulation[[
    "date_key",
    "store_key", 
    "store_cumulative_revenue",
    "store_cumulative_transactions"
]].drop_duplicates(
    subset=["date_key", "store_key"], 
    keep="last"
).reset_index(drop=True)

fact_store_accumulation = fact_store_accumulation.sort_values(
    ["store_key", "date_key"]
).reset_index(drop=True)

In [86]:
fact_store_accumulation

,date_key,store_key,store_cumulative_revenue,store_cumulative_transactions
0,20251101,1,310950,4
1,20251102,1,553800,7
2,20251103,1,1200025,12
3,20251104,1,2025525,17
4,20251105,1,2574525,19
...,...,...,...,...
83,20251126,3,15493125,109
84,20251127,3,16050050,115
85,20251128,3,16803075,120
86,20251129,3,17009575,122


In [87]:
# Export Dimensions into Parquet files
dim_date.to_parquet("dimensions/dim_date.parquet", index=False)
dim_store.to_parquet("dimensions/dim_store.parquet", index=False)
dim_product.to_parquet("dimensions/dim_product.parquet", index=False)
dim_supplier.to_parquet("dimensions/dim_supplier.parquet", index=False)
dim_customer.to_parquet("dimensions/dim_customer.parquet", index=False)
dim_cashier.to_parquet("dimensions/dim_cashier.parquet", index=False)
dim_payment.to_parquet("dimensions/dim_payment.parquet", index=False)

In [88]:
# Export Facts into Parquet files
fact_sales.to_parquet("facts/fact_sales.parquet", index=False)
fact_daily_sales.to_parquet("facts/fact_daily_sales.parquet", index=False)
fact_store_performance.to_parquet("facts/fact_store_performance.parquet", index=False)
fact_product_performance.to_parquet("facts/fact_product_performance.parquet", index=False)
fact_store_daily_balance.to_parquet("facts/fact_store_daily_balance.parquet", index=False)
fact_product_daily_movement.to_parquet("facts/fact_product_daily_movement.parquet", index=False)
fact_customer_daily_activity.to_parquet("facts/fact_customer_daily_activity.parquet", index=False)
fact_promo_coverage.to_parquet("facts/fact_promo_coverage.parquet", index=False)
fact_product_availability.to_parquet("facts/fact_product_availability.parquet", index=False)
fact_daily_snapshot.to_parquet("facts/fact_daily_snapshot.parquet", index=False)
fact_store_snapshot.to_parquet("facts/fact_store_snapshot.parquet", index=False)
fact_customer_accumulation.to_parquet("facts/fact_customer_accumulation.parquet", index=False)
fact_store_accumulation.to_parquet("facts/fact_store_accumulation.parquet", index=False)